In [1]:
import sys
sys.path.append('/gale/netapp/home/wtian/local/pyballc/pyballc/')
import glob


In [2]:
import pyballcools
import pysam

In [5]:
ballc_file = '/gale/netapp/home/wtian/local/ballcools/test/out/HBA_200616_H1930001_CX50_M1C_1_P5-3-E10-A17.ballc'
cmeta_file = '/gale/netapp/home/wtian/bican/projects/test/h1930001.cmeta.gz'

In [6]:
next(pysam.TabixFile(cmeta_file).fetch(*region))

'chr1\t10004\t+\tCCC'

In [3]:
region = 'chr1', 0,20000
# region = 'chr1', 62000, 70000
# region = 'chr1', 0,10000
region = 'chr1', 0, 80000


In [9]:
class BAllCFile:
    def __init__(self, ballc_file, cmeta_file=None):
        self.bci = pyballcools.BAllCIndex(ballc_file)
        self.tbi = pysam.TabixFile(cmeta_file) if cmeta_file is not None else None
    def _fetch_with_cmeta(self, chrom, start, end):
        mciter = self.bci.QueryMcRecords_Iter(chrom, start, end)
        if mciter.HasNext():
            mciter.Next()
        while mciter.HasNext():
            rec = mciter.Next()
            try:
                cline = next(self.tbi.fetch(rec.chrom,rec.pos-1,rec.pos,))
            except:
                ##TODO
                pass

            *_, strand, context = cline.split()

            yield(rec.chrom,rec.pos,strand, context, rec.mc,rec.cov, )
    def _fetch(self, chrom, start, end):
        mciter = self.bci.QueryMcRecords_Iter(chrom, start, end)
        if mciter.HasNext():
            mciter.Next()
        while mciter.HasNext():
            rec = mciter.Next()
            yield(rec.chrom, rec.pos, rec.mc, rec.cov, )
        
    def fetch(self, chrom, start, end):
        if self.tbi is None:
            return self._fetch(chrom, start, end)
        else:
            return self._fetch_with_cmeta(chrom, start, end)

In [10]:
ballc = BAllCFile(ballc_file, cmeta_file)

In [11]:
for x in ballc.fetch(*region):
    print(x)

('chr1', 17397, '-', 'CTT', 0, 1)
('chr1', 17400, '-', 'CAG', 0, 1)
('chr1', 17407, '-', 'CGG', 1, 1)
('chr1', 17410, '-', 'CAG', 0, 1)
('chr1', 17417, '-', 'CTC', 0, 1)
('chr1', 17443, '-', 'CTG', 0, 1)
('chr1', 17444, '-', 'CCT', 0, 1)
('chr1', 17445, '-', 'CCC', 0, 1)
('chr1', 17446, '-', 'CCC', 0, 1)
('chr1', 17451, '-', 'CTG', 0, 1)
('chr1', 17453, '-', 'CGC', 1, 1)
('chr1', 17455, '-', 'CAC', 0, 1)
('chr1', 17460, '-', 'CAG', 0, 1)
('chr1', 17462, '-', 'CAC', 1, 1)
('chr1', 17463, '-', 'CCA', 0, 1)
('chr1', 17464, '-', 'CCC', 0, 1)
('chr1', 17465, '-', 'CCC', 0, 1)
('chr1', 17471, '-', 'CTG', 0, 1)
('chr1', 17472, '-', 'CCT', 0, 1)
('chr1', 17479, '-', 'CGG', 1, 1)
('chr1', 17481, '-', 'CTC', 0, 1)
('chr1', 17484, '-', 'CGG', 1, 1)
('chr1', 17486, '-', 'CTC', 0, 1)
('chr1', 17493, '-', 'CGG', 1, 1)
('chr1', 17503, '-', 'CAG', 0, 1)
('chr1', 17504, '-', 'CCA', 0, 1)
('chr1', 17510, '-', 'CAG', 0, 1)
('chr1', 17511, '-', 'CCA', 0, 1)
('chr1', 62002, '+', 'CAG', 0, 2)
('chr1', 62006